In [1]:
import unittest

## Test Configuration

Configure the Ameritrade client to use the username, password and client id stored in environment variables.

In [2]:
import importlib
import ameritrade_functions as amc
importlib.reload(amc)
import os

os.environ['ameritradeuser'] = 'test_user'
os.environ['ameritradepw'] = 'test_pw'
os.environ['ameritradeclientid'] = 'client_id'

class TestConfiguration(unittest.TestCase):
    def test_config_default(self):
        class_under_test = amc.AmeritradeRest()
        self.assertEqual(class_under_test.username, 'test_user')
        self.assertEqual(class_under_test.password, 'test_pw')
        self.assertEqual(class_under_test.client_id, 'client_id')  
        self.assertEqual(class_under_test.consumer_key, 'client_id'+'@AMER.OAUTHAP')
    
    def test_config_parms(self):
        class_under_test = amc.AmeritradeRest('ameritradeuser', 'ameritradepw', 'ameritradeclientid')
        self.assertEqual(class_under_test.username, 'test_user')
        self.assertEqual(class_under_test.password, 'test_pw')
        self.assertEqual(class_under_test.client_id, 'client_id')
        self.assertEqual(class_under_test.consumer_key, 'client_id'+'@AMER.OAUTHAP')
        
TestConfiguration().test_config_default()
TestConfiguration().test_config_parms()

## Test Authentication

This requires you to have a developer account with Ameritrade (https://developer.tdameritrade.com).

In [24]:
import importlib
import ameritrade_functions as amc
importlib.reload(amc)
import os

# For Chromedriver
from pathlib import Path

class TestAuthentication(unittest.TestCase):
    def test_authentication(self):
        td_ameritrade = amc.AmeritradeRest('maiotradeuser', 'maiotradepw', 'maiotradeclientid')
        td_ameritrade.authenticate()
        self.assertGreater(len(td_ameritrade.authorization), 0)

TestAuthentication().test_authentication()